In [9]:
from Scraping_Bitcoin_News import get_bitcoin_news 
from alphadvantage_crypto import get_crypto_data
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

In [2]:
# make sure to load dotenv
load_dotenv()

# Database connection paramete(s
Database_URL = os.getenv('URL_Railway')

#connect to database
conn = psycopg2.connect(Database_URL)
cur = conn.cursor()

#create tables if they don't exist

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_prices(
    date DATE PRIMARY KEY,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume FLOAT
    
)            
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bitcoin_news(
    id SERIAL PRIMARY KEY,
    date DATE,
    headline TEXT,
    subheadline TEXT
    
)            
''')

conn.commit()

In [7]:
#function to insert bitcoin_data
def insert_bitcoin_data(conn, data):
    try:
        cursor = conn.cursor()
        for index, row in data.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_prices(date, open, high, low, close, volume)
                Values(%s, %s, %s, %s, %s, %s)
                ON CONFLICT (date) DO NOTHING
                ''',(index, row['open'], row['high'], row['low'], row['close'], row['volume']))
        conn.commit()
        cursor.close()
        print("sucess")
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

In [10]:
#function to insert bitcoin_news
def insert_bitcoin_news(conn, df):
    try:
        cursor = conn.cursor()
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO bitcoin_news (id, date , headline, subheadline)
                Values(%s, %s, %s, %s)
                ON CONFLICT (id) DO NOTHING
                ''',(index,row['date'], row['headline'], row['subheadline']))
        conn.commit()
        cursor.close()
    except Exception as e:
        print(f'Error inserting Bitcoin data: {e}')
        conn.rollback()

In [11]:
#Main Function to orchestrate the data fetching and insertion
def main():
    # fetch Bitcoin Data
    bitcoin_data = get_crypto_data('BTC','USD')

    #Scrape Bitcoin News data
    bitcoin_news = get_bitcoin_news()

    #connect to postgresSQL database
    conn = psycopg2.connect(Database_URL)

    if conn:
        #Insert data into database
        insert_bitcoin_data(conn,bitcoin_data)
        insert_bitcoin_news(conn,bitcoin_news)

        #close the database connection
        conn.close()
    else:
        print('Failed to connect to the database')

if __name__== '__main__':
    main()


sucess
